<a href="https://colab.research.google.com/github/ssaltwick/ENEE324-Project/blob/master/ENEE324_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


This program uses data from the 2014-2018 NBA seasons to classify players by 
position.


Previous year's game statistics are used to generate a likelihood function
for each position. Then a certain player can be tested against each of these 
functions to guess their position.


In [0]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stat
import math
import pandas as pd
from enum import Enum
from mpl_toolkits import mplot3d
from itertools import combinations

from sklearn.mixture import GaussianMixture



In [0]:

# Defines an enumeration for player positions

class Position(Enum):
  PG = 0
  SG = 1
  SF = 2
  PF = 3
  C = 4
  

In [0]:
# fig = plt.figure()
# ax = plt.axes(projection='3d')

# ax.scatter3D(data[Position.PG]['3PA'],data[Position.PG]['2P%'],data[Position.PG]['BLK'])
# # your ellispsoid and center in matrix form
# A = covs[Position.PG]
# center = avgs[Position.PG]

# # find the rotation matrix and radii of the axes
# U, s, rotation = np.linalg.svd(A)
# radii = 1.0/np.sqrt(s)

# # now carry on with EOL's answer
# u = np.linspace(0.0, 2.0 * np.pi, 100)
# v = np.linspace(0.0, np.pi, 100)
# x = radii[0] * np.outer(np.cos(u), np.sin(v))
# y = radii[1] * np.outer(np.sin(u), np.sin(v))
# z = radii[2] * np.outer(np.ones_like(u), np.cos(v))
# for i in range(len(x)):
#     for j in range(len(x)):
#         [x[i,j],y[i,j],z[i,j]] = np.dot([x[i,j],y[i,j],z[i,j]], rotation) + center



# ax.plot_wireframe(x, y, z,  rstride=4, cstride=4, color='b', alpha=0.2)
# plt.show()
# plt.close(fig)
# del fig



In [0]:
"""
  Evaluates a player against a position's likelihood.
  params: positon = Position Enum
          player = numpy array of stats

"""
def evaluate_likelihood(position, player, avgs, covs, prior):
  mu = avgs[position]
  sig = covs[position]
  
  p = (math.sqrt(2*math.pi)) ** 3
  c = 1 / (p * math.sqrt(np.linalg.det(sig)))
  
  # print(sig.shape, mu.shape, player.shape)
   
  t = np.dot(np.transpose(player-mu), np.dot(sig, (player-mu)))
  
  
  return c * math.exp(-0.5 * t)

In [0]:
def guess_position(player, avgs, covs, prior, pos):
  positions = {}
  
  for p in pos:
    positions[p] = evaluate_likelihood(p, player, avgs, covs, prior) * prior[p]
  
  v = list(positions.values())
  k = list(positions.keys())
  return k[v.index(max(v))]

In [0]:
def compare_position(actual, guessed):

  pos_names = {
      Position.PG : "PG",
      Position.SG : "SG",
      Position.SF : "SF",
      Position.PF : "PF",
      Position.C : "C",
  }

# Guard, Forward, Center

#   if guessed == Position.PG or guessed == Position.SG:
#     if actual == "PG" or actual == "SG":
#       return 1
#   if guessed == Position.SF or guessed == Position.PF:
#     if actual == "SF" or actual == "PF":
#       return 1
#   if guessed == Position.C:
#     if actual == "C":
#       return 1
#   return 0

# (PG, SG, SF) or (PF, C)
#   if guessed == Position.PG or guessed == Position.SG or guessed == Position.SF:
#     if actual == "PG" or actual == "SG" or actual == "SF":
#       return 1
#   if guessed == Position.C or guessed == Position.PF:
#     if actual == "SF" or actual == "PF":
#       return 1
#   return 0

# (PG, SG), SF, (PF, C)
#   if guessed == Position.PG or guessed == Position.SG:
#     if actual == "PG" or actual == "SG":
#       return 1
#   if guessed == Position.C or guessed == Position.PF:
#     if actual == "C" or actual == "PF":
#       return 1
#   if guessed == Position.SF:
#     if actual == "SF":
#       return 1
#   return 0

# Just SF
# if guess == Position.SF:
#   if actual == "SF":
#     return 1
# return 0


# PG, SG, SF, PF, C
  if pos_names[guessed] == actual:
    # print("Position %s Guessed Correctly" % actual)
    return 1
  else:
    # print("Position Guessed Incorrectly- Actual: %s     Guessed: %s" %(actual, pos_names[guessed]))
    return 0
      


In [0]:
# Load Data
pg_url = 'https://raw.githubusercontent.com/ssaltwick/ENEE324-Project/master/data/Data%20-%20PG-clean.csv'
sg_url = 'https://raw.githubusercontent.com/ssaltwick/ENEE324-Project/master/data/Data%20-%20SG-clean.csv'
sf_url = 'https://raw.githubusercontent.com/ssaltwick/ENEE324-Project/master/data/Data%20-%20SF-clean.csv'
pf_url = 'https://raw.githubusercontent.com/ssaltwick/ENEE324-Project/master/data/Data%20-%20PF-clean.csv'
c_url = 'https://raw.githubusercontent.com/ssaltwick/ENEE324-Project/master/data/Data%20-%20C-Clean.csv'
test_url = 'https://raw.githubusercontent.com/ssaltwick/ENEE324-Project/master/data/Data%20-%20Test-Clean.csv'


urls = {
    Position.PG : pg_url,
    Position.SG : sg_url,
    Position.SF : sf_url,
    Position.PF : pf_url,
    Position.C : c_url
}
test_data = pd.read_csv(test_url).dropna()
#2p%, 3pa, ft%


# Selected Stats (Best so far)
#stats = ['3PA', '3P%', '2P%',  'BLK']

In [0]:
def find_best_stats(num_stats):
  # All stats
  all_stats = ['3PA', '3P%','2P%', 'PTS', 'AST', 'FT%', 'STL', 'BLK', 'ORB','TRB']
  combos = list(combinations(all_stats,num_stats))

  results = {}
  for stats in combos:
    stats = list(stats)
    data = {
        Position.PG : pd.read_csv(pg_url).dropna()[stats],
        Position.SG : pd.read_csv(sg_url).dropna()[stats],
        Position.SF : pd.read_csv(sf_url).dropna()[stats],
        Position.PF : pd.read_csv(pf_url).dropna()[stats],
        Position.C : pd.read_csv(c_url).dropna()[stats]
    }

    total_players = data[Position.PG].shape[0] + data[Position.SG].shape[0] + \
      data[Position.SF].shape[0] + data[Position.PF].shape[0] + data[Position.C].shape[0]


    # Defines an array of priors (each position's percentage of players)
    # TODO: Update with actual priors),
    prior = {
        Position.PG : data[Position.PG].shape[0] / total_players,
        Position.SG : data[Position.SG].shape[0] / total_players,
        Position.SF : data[Position.SF].shape[0] / total_players,
        Position.PF : data[Position.PF].shape[0] / total_players,
        Position.C : data[Position.C].shape[0] / total_players
    }




    # TODO: Generate actual MEAN and COV for each position

    avgs = {
        Position.PG : data[Position.PG].mean(0).to_numpy(),
        Position.SG : data[Position.SG].mean(0).to_numpy(),
        Position.SF : data[Position.SF].mean(0).to_numpy(),
        Position.PF : data[Position.PF].mean(0).to_numpy(),
        Position.C : data[Position.C].mean(0).to_numpy()
    }

    covs = {
        Position.PG : data[Position.PG].cov().to_numpy(),
        Position.SG : data[Position.SG].cov().to_numpy(),
        Position.SF : data[Position.SF].cov().to_numpy(),
        Position.PF : data[Position.PF].cov().to_numpy(),
        Position.C : data[Position.C].cov().to_numpy()
    }

    # Display stat with minimum variance 
    # print(np.argmin(np.diag(covs[Position.PG])))
    # print(np.argmin(np.diag(covs[Position.SG])))
    # print(np.argmin(np.diag(covs[Position.SF])))
    # print(np.argmin(np.diag(covs[Position.PF])))
    # print(np.argmin(np.diag(covs[Position.C])))






    test_frame = test_data[['Pos'] + stats]

    sample_size = test_frame.shape[0]
    num_correct = 0.0


    for i in range(0,sample_size):

      test_player = test_frame.to_numpy()[i,0:]

      guess = guess_position(test_player[1:], avgs, covs, prior)

      num_correct += compare_position(test_player[0], guess)

    percent_correct = (num_correct / sample_size)


    results[''.join(stats)] = percent_correct

    #print(''.join(stats))
    #print('Guessed {:.2%}  of players correct'.format(percent_correct))



  v = list(results.values())
  k = list(results.keys())

  best = k[v.index(max(v))]  
  print("Best result using %s with %s accuracy" % (best, max(v)))

In [0]:
def compare_positions(num_stats, compare):
  all_stats = ['3PA', '3P%','2P%', 'PTS', 'AST', 'FT%', 'STL', 'BLK', 'ORB','TRB']
  combos = list(combinations(all_stats,num_stats))
  
  pos_names = {
    Position.PG : "PG",
    Position.SG : "SG",
    Position.SF : "SF",
    Position.PF : "PF",
    Position.C : "C",
  }
  
  results = {}
  for stats in combos:
    stats = list(stats)
    data = {}
    
    for p in compare:
      data[p] = pd.read_csv(urls[p]).dropna()[stats]  
      
      

    total_players = 0
    for p in compare:
      total_players += data[p].shape[0]
    


    # Defines an array of priors (each position's percentage of players)
    # TODO: Update with actual priors),

    prior = {}
    avgs = {}
    covs = {}
    


    # TODO: Generate actual MEAN and COV for each position


    for p in compare:
      total_players += data[p].shape[0]
    
    for p in compare:
      prior[p] = data[p].shape[0] / total_players
      
    for p in compare:
      avgs[p] = data[p].mean(0).to_numpy()
      covs[p] = data[p].cov().to_numpy()
      
    
    # Display stat with minimum variance 
    # print(np.argmin(np.diag(covs[Position.PG])))
    # print(np.argmin(np.diag(covs[Position.SG])))
    # print(np.argmin(np.diag(covs[Position.SF])))
    # print(np.argmin(np.diag(covs[Position.PF])))
    # print(np.argmin(np.diag(covs[Position.C])))


    test_frame = test_data[['Pos'] + stats]

    fd = False
    for p in compare:
      fd |= test_frame['Pos'] == pos_names[p]
  
    test_frame = test_frame.loc[fd]
    sample_size = test_frame.shape[0]
    num_correct = 0.0


    for i in range(0,sample_size):

      test_player = test_frame.to_numpy()[i,0:]

      guess = guess_position(test_player[1:], avgs, covs, prior, compare)

      num_correct += compare_position(test_player[0], guess)

    percent_correct = (num_correct / sample_size)


    results[''.join(stats)] = percent_correct

    #print(''.join(stats))
    #print('Guessed {:.2%}  of players correct'.format(percent_correct))



  v = list(results.values())
  k = list(results.keys())

  best = k[v.index(max(v))]  
  print("Best result comparing %s using %s with %s accuracy" % (''.join(str(compare)), best, max(v)))

In [0]:
# positions = [Position.PG, Position.SG, Position.SF, Position.PF, Position.C]

# for i in range (1,6):
#   combos = list(combinations(positions,i))
  
#   for c in combos[5:]:
#     compare_positions(4, c)



In [0]:
# for i in range(1,11):
#   compare_positions(i, [Position.PG, Position.SG, Position.SF])

In [0]:
def calculate_correct(bins, total):
  num_correct = 0
  for k in bins.keys():
    for p in bins[k]:
      if p[1] in k:
        num_correct += 1
        
  return (num_correct / total)

In [0]:
def stage1(round1_stats, labels):
  
  pos_names = {
      Position.PG : "PG",
      Position.SG : "SG",
      Position.SF : "SF",
      Position.PF : "PF",
      Position.C : "C",
  }
  
  b1 = [Position.PG, Position.SG, Position.SF]
  b2 = [Position.PF, Position.C]
  
  

  
  positions = [Position.PG, Position.SG, Position.SF, Position.PF, Position.C]
  

  bins = {}
  
  for l in labels:
    tmp = [pos_names[x] for x in l]
    bins[''.join(tmp)] = []
    
  
  data = {}
  for p in positions:
    data[p] = pd.read_csv(urls[p]).dropna()[round1_stats]  



  total_players = 0
  for p in positions:
    total_players += data[p].shape[0]



  # Defines an array of priors (each position's percentage of players)
  # TODO: Update with actual priors),

  prior = {}
  avgs = {}
  covs = {}


  # TODO: Generate actual MEAN and COV for each position


  for p in positions:
    total_players += data[p].shape[0]

  for p in positions:
    prior[p] = data[p].shape[0] / total_players

  for p in positions:
    avgs[p] = data[p].mean(0).to_numpy()
    covs[p] = data[p].cov().to_numpy()


  test_frame = test_data[['Player'] + ['Pos'] + round1_stats]

  sample_size = test_frame.shape[0]



  for i in range(0,sample_size):

    test_player = test_frame.to_numpy()[i,0:]
    
    guess = guess_position(test_player[2:], avgs, covs, prior, positions)
    
    
    for l in labels:
      tmp = [pos_names[x] for x in l]
      k = ''.join(tmp)
      if guess in l:
        bins[k].append(test_player)
       

  percent_correct = calculate_correct(bins, sample_size)
      
  print("Stage 1 clustered {:.2%} correctly".format(percent_correct))
  
  return percent_correct, bins
  
  

In [0]:
def stage2(labels, stats, names):
  
  positions = [Position.PG, Position.SG, Position.SF, Position.PF, Position.C]
  
  bins = {}
  for p in labels:
    bins[p] = []
  
  pos_names = {
      Position.PG : "PG",
      Position.SG : "SG",
      Position.SF : "SF",
      Position.PF : "PF",
      Position.C : "C",
  }
  
  
  data = {}
  for p in positions:
    data[p] = pd.read_csv(urls[p]).dropna()[stats]  


  total_players = 0
  for p in positions:
    total_players += data[p].shape[0]



  # Defines an array of priors (each position's percentage of players)
  # TODO: Update with actual priors),

  prior = {}
  avgs = {}
  covs = {}


  # TODO: Generate actual MEAN and COV for each position


  for p in positions:
    total_players += data[p].shape[0]

  for p in positions:
    prior[p] = data[p].shape[0] / total_players

  for p in positions:
    avgs[p] = data[p].mean(0).to_numpy()
    covs[p] = data[p].cov().to_numpy()
    
    
    
  items = pd.read_csv(test_url).dropna()[['Player', 'Pos'] + stats]
  items = items[items['Player'].isin(names)]
  
  items = items.values

  sample_size = items.shape[0]
  
  

  for i in range(0,sample_size):

    test_player = items[i]
    
    guess = guess_position(test_player[2:], avgs, covs, prior, labels)

    bins[guess].append(test_player)

  num_correct = 0
  for k in bins.keys():
    for v in bins[k]:
      if v[1] == pos_names[k]:
        num_correct += 1
  
  
  percent_correct = num_correct / sample_size;
      
  # print("Stage 2 clustered {:.2%} correctly".format(percent_correct))
  
  return percent_correct, bins

In [0]:
# Find best stage 1 stats

def find_best_s1_stats():
  used_pos = [[Position.PG, Position.SG],[Position.SF], [Position.PF, Position.C]]
  round1_stats = ['3PA', '3P%', '2P%', 'BLK']
  all_stats = ['3PA', '3P%','2P%', 'PTS', 'AST', 'FT%', 'STL', 'BLK', 'ORB','TRB']



  for i in range (1,11):
    combos = list(combinations(all_stats,i))

    m = (0,0)
    for c in combos:
      pc, _ = stage1(list(c), used_pos)
      if pc > m[0]:
        m = (pc, c)

    print(m)


In [0]:
find_best_s1_stats()

In [0]:
def accuracy(splits):
  num_correct = 0
  total = 0

  for split in splits.values():
    for k in split.keys():
      for v in split[k]:
        if v[1] == pos_names[k]:
          num_correct += 1
        total += 1
      
  return num_correct/total

In [0]:
# BEST SO FAR
# PGSGSF / PFC
# ['3P%', 'AST', 'FT%', 'STL', 'BLK', 'ORB'], ['3PA', 'BLK']


used_pos = [[Position.PG, Position.SG],[Position.SF], [Position.PF, Position.C]]
best_stats = [['3P%', 'AST', 'FT%', 'STL', 'BLK', 'ORB'], ['3P%', 'AST', 'FT%', 'STL', 'BLK', 'ORB'], ['3PA', 'BLK']]
stage1_stats = ['3PA', '2P%', 'TRB']


pc, first_split, a, p, c = stage1(stage1_stats, used_pos)

all_stats = ['3PA', '3P%','2P%', 'PTS', 'AST', 'FT%', 'STL', 'BLK', 'ORB','TRB']
stats = []


pos_names = {
      Position.PG : "PG",
      Position.SG : "SG",
      Position.SF : "SF",
      Position.PF : "PF",
      Position.C : "C",
  }
 
s = []
for k in first_split.keys():
  s.append(first_split[k])

names = []

for i in range(len(s)):
  names.append([s[0] for s in s[i]])
  



for i in range(len(best_stats)):
  stats.append(best_stats[i])
    
    
    
    
splits = {}    
# i = 0
# for k in names.keys():
#   _, splits[k] = stage2(used_pos[i], stats[i], names[i])
#   i += 1
  
# acc = accuracy(splits)


# print("Predicted {:2%} of player's positions correctly".format(acc))

# Find best # of stats
for i in range(1,9):
  combos = list(combinations(all_stats,i))
  m = (0,0)
  for c in combos:
    pc, sec_split2 = stage2(used_pos[1], list(c), names[1])
    if pc > m[0]:
      m = (pc, c)

  print(m)

